In [49]:
import os
import joblib
from utils import pretty_trim, simple_split
from collections import Counter
from scipy.sparse import vstack
import numpy as np

** Loading trained models **

In [63]:
%%time
filename = 'models_persistence/pickle_models'
(pretty_trim, counter, tfidf, rfe, clfs) = joblib.load(filename)

Wall time: 3.59 s


In [19]:
%%time
filename = 'models_persistence/final_dataset'
(X_train_final, y_train, X_test_final, y_test) = joblib.load(filename)

Wall time: 1.66 s


In [62]:
# # refitting on the whole dataset
# X_old, y_old = vstack([X_train_final, X_test_final]), np.concatenate([y_train, y_test])
# clf.fit(X_old, y_old)

In [64]:
clf = clfs[0]
print 'score:', clf.score(X_test_final, y_test)
proba = clf.predict_proba(X_test_final)
confidence = proba.max(1)
print 'confidence:', confidence.mean(), confidence.std(), confidence.min(), confidence.max()

score: 0.927450980392
confidence: 0.73368627451 0.220944806363 0.16 1.0


** Read segmented docs **

In [7]:
%%time
doc_path = u'./corpus/segmented-journal' # must be a segmented doc path
dataset_contents = []
for filename in os.listdir(doc_path):
    path = os.path.join(doc_path, filename)
    with open(path) as f:
        content = f.read().decode('utf8')
        dataset_contents.append(content)
print 'total files:', len(dataset_contents)

total files: 2165
Wall time: 394 ms


# Apply learning pipeline to all the docs
First trim

In [22]:
%%time
for i in xrange(len(dataset_contents)):
    dataset_contents[i] = pretty_trim(dataset_contents[i])

Wall time: 5.57 s


Then count words and apply Tf-idf

In [26]:
%time X_new_count = counter.transform(dataset_contents)
%time X_new_tfidf = tfidf.transform(X_new_count)
print X_new_tfidf.shape

Wall time: 17.3 s
Wall time: 200 ms
(2165, 250000)


Remove some features

In [27]:
%time X_new_rfe = rfe.transform(X_new_tfidf)
print X_new_rfe.shape

Wall time: 82 ms
(2165, 20000)


Predict using trained models

In [65]:
y_pred = clf.predict(X_new_rfe)
Counter(y_pred)

Counter({0: 1,
         2: 1,
         3: 1,
         5: 1322,
         7: 15,
         9: 32,
         12: 2,
         15: 192,
         16: 575,
         17: 17,
         19: 7})

In [66]:
proba = clf.predict_proba(X_new_rfe)
confidence = proba.max(1)
print 'confidence:', confidence.mean(), confidence.std(), confidence.min(), confidence.max()

confidence: 0.457131639723 0.130549181945 0.13 0.92
